In [1]:
%gui qt
%load_ext autoreload
%autoreload 2
    
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.Bnd import Bnd_OBB, Bnd_Box
from OCC.Core.BRepBuilderAPI import (
    BRepBuilderAPI_Transform,
    BRepBuilderAPI_MakeEdge,
    BRepBuilderAPI_MakeWire,
    BRepBuilderAPI_MakeFace,
)
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.GProp import GProp_GProps
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE
from OCC.Core.TopoDS import TopoDS_Compound, TopoDS_Wire
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.gp import gp_Trsf, gp_Pnt, gp_Ax3, gp_Ax1, gp_Dir, gp_Pln, gp_Vec, gp_Mat, gp_Quaternion
from OCC.Display.SimpleGui import init_display
from OCC.Core.TopLoc import TopLoc_Location
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from IPython.display import display
from IPython.display import display, HTML

import pandas as pd
import numpy as np
import json
import math

# Start display without blocking
viewer, start_display, add_menu, add_function_to_menu = init_display()

# JSON Library path
json_path = "../data/Shape_classifier_info.json"

viewer.EraseAll()
viewer.View_Iso()
viewer.FitAll()

pyside6 backend - Qt version 6.8.3


In [2]:
# Load the STEP and extract 'solid'
def load_step(file_path: str):
    reader = STEPControl_Reader()
    status = reader.ReadFile(file_path)
    if status != 1:
        raise RuntimeError(f"Failed to read STEP file: {file_path}")
    reader.TransferRoots()
    return reader.Shape()

In [3]:
def show_origin():

# Draw global axes
    origin = gp_Pnt(0, 0, 0)
    scale = 50

    # Axis lines
    axes = [
        ((scale, 0, 0), "RED"),
        ((0, scale, 0), "GREEN"),
        ((0, 0, scale), "BLUE")
    ]

    for vec, axis_color in axes:
        end = gp_Pnt(*vec)
        edge = BRepBuilderAPI_MakeEdge(origin, end).Edge()
        viewer.DisplayShape(edge, color=axis_color, update=False)

In [4]:
# # Rotate z-axis 90 degrees

# def rotate_z90(shape_to_rotate):
#     angle_rad = math.radians(90)
#     axis = gp_Ax1(gp_Pnt(0, 0, 0), gp_Dir(0, 0, 1))  # Z-axis through origin
#     trsf_rotate_z = gp_Trsf()
#     trsf_rotate_z.SetRotation(axis, angle_rad)
#     rotated_shape = BRepBuilderAPI_Transform(shape_to_rotate, trsf_rotate_z, True).Shape()
#     return rotated_shape

In [5]:
def visualize(shape, display, color="CYAN", clear=False):
    """
    Displays a shape with global XYZ axes and OBB extents.
    
    Args:
        shape: The TopoDS_Shape to display.
        display: OCC display handle (e.g., from init_display()).
        color: Optional color for the shape.
        clear: If True, erase all previous shapes before displaying.
    """

    def get_occ_color(color_name):
        colors = {
            "RED": (1.0, 0.0, 0.0),
            "GREEN": (0.0, 1.0, 0.0),
            "BLUE": (0.0, 0.0, 1.0),
            "PSS_DARK_BLUE": (0.023, 0.105, 0.215),
            "PSS_LIGHT_BLUE": (0.592, 0.792, 0.921),
            "YELLOW": (1.0, 1.0, 0.0),
            "ORANGE": (1.0, 0.5, 0.0),
            "CYAN": (0.0, 1.0, 1.0),
            "MAGENTA": (1.0, 0.0, 1.0),
            "WHITE": (1.0, 1.0, 1.0),
            "GRAY": (0.5, 0.5, 0.5),
            "BLACK": (0.0, 0.0, 0.0),
        }
        r, g, b = colors.get(color_name.upper(), (1.0, 1.0, 1.0))
        return Quantity_Color(r, g, b, Quantity_TOC_RGB)
    
    # if clear:
        viewer.EraseAll()

    # Convert color name to Quantity_Color
    qcolor = get_occ_color(color)

    # Display the shape
    viewer.DisplayShape(shape, color=qcolor, update=False)

    viewer.FitAll()



In [6]:
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

def draw_obb_box(obb, viewer, color="ORANGE", update=True):
    """
    Draws the 12 edges of the OBB as lines in the viewer.
    """
    c = obb['center']
    cx, cy, cz = c.X(), c.Y(), c.Z()
    hx, hy, hz = obb['half_x'], obb['half_y'], obb['half_z']
    dx, dy, dz = obb['x_dir'], obb['y_dir'], obb['z_dir']

    # Precompute corner offsets
    offsets = [
        (+hx, +hy, +hz), (-hx, +hy, +hz), (-hx, -hy, +hz), (+hx, -hy, +hz),
        (+hx, +hy, -hz), (-hx, +hy, -hz), (-hx, -hy, -hz), (+hx, -hy, -hz)
    ]

    # Compute corners
    corners = []
    for ox, oy, oz in offsets:
        x = cx + ox * dx.X() + oy * dy.X() + oz * dz.X()
        y = cy + ox * dx.Y() + oy * dy.Y() + oz * dz.Y()
        z = cz + ox * dx.Z() + oy * dy.Z() + oz * dz.Z()
        corners.append(gp_Pnt(x, y, z))

    # Define edges by corner indices
    edges_idx = [
        (0, 1), (1, 2), (2, 3), (3, 0),  # top face
        (4, 5), (5, 6), (6, 7), (7, 4),  # bottom face
        (0, 4), (1, 5), (2, 6), (3, 7),  # vertical edges
    ]

    for i1, i2 in edges_idx:
        edge = BRepBuilderAPI_MakeEdge(corners[i1], corners[i2]).Edge()
        viewer.DisplayShape(edge, color=color, update=False)

    if update:
        viewer.FitAll()

In [7]:
# face alignemnt worst case fix, no z/x/y alignment.
# Find the largest face by area → assume it defines primary orientation.
# Align its longest edge to the Z axis.
# Find a second large face roughly orthogonal to that → use to resolve X or Y.

from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE, TopAbs_EDGE
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve, BRepAdaptor_Surface
from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Vec, gp_Ax3, gp_Trsf
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform


def robust_align_solid_from_geometry(solid, tol=1e-3):
    """
    Align a solid using its geometry:
    - Z axis is taken from the longest edge of the largest planar face
    - X/Y axes are derived from a second large face roughly perpendicular to Z
    Returns: aligned solid, transformation, and reference Ax3
    """
    # Step 1: Find the largest planar face
    explorer = TopExp_Explorer(solid, TopAbs_FACE)
    largest_face = None
    max_area = 0.0
    face_areas = []

    while explorer.More():
        face = explorer.Current()
        props = GProp_GProps()
        brepgprop.SurfaceProperties(face, props)
        area = props.Mass()
        face_areas.append((face, area))
        if area > max_area:
            max_area = area
            largest_face = face
        explorer.Next()

    if not largest_face:
        raise RuntimeError("No faces found in solid.")

    # Step 2: Get longest edge of that face
    edge_exp = TopExp_Explorer(largest_face, TopAbs_EDGE)
    longest_edge = None
    max_length = 0.0
    longest_vec = None

    while edge_exp.More():
        edge = edge_exp.Current()
        curve_handle, u1, u2 = BRep_Tool.Curve(edge)
        curve = BRepAdaptor_Curve(edge)
        p1 = curve.Value(curve.FirstParameter())
        p2 = curve.Value(curve.LastParameter())
        vec = gp_Vec(p1, p2)
        length = vec.Magnitude()
        if length > max_length:
            max_length = length
            longest_edge = edge
            longest_vec = vec
        edge_exp.Next()

    if not longest_vec:
        raise RuntimeError("No edge found on largest face.")

    z_dir = gp_Dir(longest_vec)

    # Step 3: Find a second large face with normal ≠ Z to define X
    second_face = None
    second_normal = None
    for face, area in sorted(face_areas, key=lambda x: -x[1]):
        if face.IsSame(largest_face):
            continue
        u, v = 0.5, 0.5
        surf_adapt = BRepAdaptor_Surface(face)
        normal = surf_adapt.Plane().Axis().Direction() if surf_adapt.GetType() == 0 else None
        if normal and abs(normal.Dot(z_dir)) < 0.9:  # roughly perpendicular
            second_face = face
            second_normal = normal
            break

    if not second_normal:
        raise RuntimeError("No suitable secondary face found for X/Y orientation.")

    x_dir = second_normal
    y_dir = z_dir.Crossed(x_dir)

    # Step 4: Create transformation from current CS to global
    origin = gp_Pnt(0, 0, 0)
    from_cs = gp_Ax3(origin, z_dir, x_dir)
    to_cs = gp_Ax3(origin, gp_Dir(0, 0, 1), gp_Dir(1, 0, 0))

    trsf = gp_Trsf()
    trsf.SetDisplacement(from_cs, to_cs)

    aligned = BRepBuilderAPI_Transform(solid, trsf, True).Shape()
    return aligned, trsf, to_cs

In [8]:
# def compute_obb(shape):
#     """
#     Compute the Oriented Bounding Box for a shape.
#     Returns a Bnd_OBB instance and geometry data
#     """
#     obb = Bnd_OBB()
#     brepbndlib.AddOBB(shape, obb)
#     # brepbndlib_AddOBB(shape, obb, use_triangulation=True, is_optimal=True, loc=TopLoc_Location())

#     #determine dominiant length for axis
#     half_extents = [
#     obb.XHSize(),
#     obb.YHSize(),
#     obb.ZHSize()
#     ]

#     length_idx = max(range(3), key=lambda i: half_extents[i])
    
#     obb_geom = {
#         'center': obb.Center(),
#         'x_dir': obb.XDirection(),
#         'y_dir': obb.YDirection(),
#         'z_dir': obb.ZDirection(),
#         'half_x': obb.XHSize(),
#         'half_y': obb.YHSize(),
#         'half_z': obb.ZHSize(),
#         'length_idx': length_idx
#     }
#     return obb, obb_geom

In [9]:
# def draw_obb_axes(obb, viewer, scale=500):
#     """
#     Visualize the X, Y, Z axes of the OBB in the viewer as arrows.
#     """
#     from OCC.Core.gp import gp_Pnt
#     from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

#     origin_xyz = obb.Center()
#     origin = gp_Pnt(origin_xyz.X(), origin_xyz.Y(), origin_xyz.Z())

#     axes = [
#         (obb.XDirection(), "RED"),
#         (obb.YDirection(), "GREEN"),
#         (obb.ZDirection(), "BLUE"),
#     ]

#     for dir_vec, color in axes:
#         end = gp_Pnt(
#             origin.X() + scale * dir_vec.X(),
#             origin.Y() + scale * dir_vec.Y(),
#             origin.Z() + scale * dir_vec.Z()
#         )
#         edge = BRepBuilderAPI_MakeEdge(origin, end).Edge()
#         viewer.DisplayShape(edge, color=color, update=False)

#     viewer.FitAll()


In [10]:
def fingerprint_shape(shape, obb):
    """
    Compute width, height, and CSA of the cross-section.
    Attempts wire-based CSA first; falls back to building from edges if needed.
    """

    half_sizes = [obb['half_x'], obb['half_y'], obb['half_z']]
    length_idx = max(range(3), key=lambda i: half_sizes[i])
    dims = [2 * h for h in half_sizes]
    width, height = sorted([dims[i] for i in range(3) if i != length_idx], reverse=True)

    center_xyz = obb['center']
    center = gp_Pnt(center_xyz.X(), center_xyz.Y(), center_xyz.Z())
    dir_map = [obb['x_dir'], obb['y_dir'], obb['z_dir']]
    normal = dir_map[length_idx]
    plane = gp_Pln(center, normal)

    section = BRepAlgoAPI_Section(shape, plane, True)
    section.ComputePCurveOn1(True)
    section.Approximation(True)
    section.Build()

    compound = section.Shape()

    # Try wire-based approach first
    exp = TopExp_Explorer(compound, TopAbs_WIRE)
    if exp.More():
        wire = exp.Current()
    else:
        # Fallback: build wire from edges
        edge_exp = TopExp_Explorer(compound, TopAbs_EDGE)
        wire_builder = BRepBuilderAPI_MakeWire()
        edge_count = 0
        while edge_exp.More():
            edge = edge_exp.Current()
            wire_builder.Add(edge)
            edge_exp.Next()
            edge_count += 1

        if edge_count == 0:
            raise RuntimeError("No wire or edge found in section result")
        if not wire_builder.IsDone():
            raise RuntimeError("Failed to build wire from edges")

        wire = wire_builder.Wire()

    # Build face from wire and compute area
    face = BRepBuilderAPI_MakeFace(plane, wire, True).Face()
    props = GProp_GProps()
    brepgprop.SurfaceProperties(face, props)
    area = props.Mass()
    centroid = props.CentreOfMass()
    return width, height, area, face, centroid

In [11]:
def build_classification_input(obb, width, height, area):
    """
    Constructs a classification-friendly dictionary using globally aligned OBB.
    Assumes the solid is aligned with Z as the length direction.
    """
    length = 2 * obb['half_z']
    return {
        "span_web": height,
        "span_flange": width,
        "area": area,
        "length": length,
    }

In [12]:
# classify shape to JSON profile list
def classify_profile(cs, json_path, tol_dim=1.0, tol_area=0.05):
    """
    Attempts to classify a structural profile by matching its dimensions and area.
    Tries both the original and swapped width/height to handle OBB axis flips.
    """
    with open(json_path) as f:
        lib = json.load(f)
    def try_match(height, width):
        best, best_score = None, float('inf')
        for cat, ents in lib.items():
            for name, info in ents.items():
                dh = abs(height - info["height"])
                dw = abs(width  - info["width"])
                if dh > tol_dim or dw > tol_dim:
                    continue
                ea = abs(cs["area"] - info["csa"]) / info["csa"]
                if ea > tol_area:
                    continue
                el = abs(cs["length"] - info.get("length", cs["length"])) / info.get("length", cs["length"])
                score = dh + dw + ea * 100 + el * 100
                if score < best_score:
                    best_score = score
                    best = {
                        **info,
                        "Designation": name,
                        "Category": cat,
                        "Measured_height": height,
                        "Measured_width": width,
                        "Measured_area": cs["area"],
                        "Measured_length": cs["length"],
                        "Match_score": score,
                        "Profile_type": info["code_profile"]
                    }
        return best, best_score

        print(f"Best: {best}")
        print(f"Best_score: {best_score}")
    
    # Try both normal and swapped width/height
    original, score1 = try_match(cs["span_web"], cs["span_flange"])
    swapped, score2  = try_match(cs["span_flange"], cs["span_web"])

    if original and (not swapped or score1 <= score2):
        original["Swapped_dimensions"] = False
        return original
    elif swapped:
        swapped["Swapped_dimensions"] = True
        return swapped
    else:
        return None

In [13]:
# # OBB transform construction
# def obb_transform(obb_geom):

#     # Create rotation matrix from OBB axes
#     rotation_matrix = gp_Mat(obb_geom['x_dir'].X(), obb_geom['y_dir'].X(), obb_geom['z_dir'].X(),
#                              obb_geom['x_dir'].Y(), obb_geom['y_dir'].Y(), obb_geom['z_dir'].Y(),
#                              obb_geom['x_dir'].Z(), obb_geom['y_dir'].Z(), obb_geom['z_dir'].Z())
    
#     # Invert the rotation to align with global axes
#     rotation_matrix.Invert()
    
#     # Create quaternion from the rotation matrix
#     quat = gp_Quaternion()
#     quat.SetMatrix(rotation_matrix)
    
#     # Create transformation from quaternion
#     trsf = gp_Trsf()
#     trsf.SetRotation(quat)

#     return trsf

In [14]:
def print_result_table(result, precision=2):
    """
    Pretty print classification result as a one-row DataFrame with rounded floats.
    """
    if not result:
        print("No classification result.")
        return

    df = pd.DataFrame([result])

    # Round float values
    for col in df.select_dtypes(include=['float']).columns:
        df[col] = df[col].round(precision)

    display(df.transpose().rename(columns={0: "Value"}))


In [15]:
from OCC.Core.gp import gp_Pnt, gp_Vec
from OCC.Display.SimpleGui import init_display
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

def draw_axis_arrow(viewer, origin, direction, scale=500, name=""):
    end_point = gp_Pnt(
        origin.X() + direction.X() * scale,
        origin.Y() + direction.Y() * scale,
        origin.Z() + direction.Z() * scale
    )
    edge = BRepBuilderAPI_MakeEdge(origin, end_point).Edge()
    viewer.DisplayShape(edge, update=False)

In [16]:
def compute_axis_aligned_bounding_box(shape):
    bbox = Bnd_Box()
    brepbndlib.Add(shape, bbox)
    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()
    
    # Half-sizes
    hx = (xmax - xmin) / 2
    hy = (ymax - ymin) / 2
    hz = (zmax - zmin) / 2
    
    center = gp_Pnt((xmin + xmax)/2, (ymin + ymax)/2, (zmin + zmax)/2)
    ax3 = gp_Ax3(center, gp_Dir(0, 0, 1), gp_Dir(1, 0, 0))
    
    return ax3, {'center': center, 'x_dir': gp_Dir(1, 0, 0), 'y_dir': gp_Dir(0, 1, 0), 'z_dir': gp_Dir(0, 0, 1),
                 'half_x': hx, 'half_y': hy, 'half_z': hz}


In [17]:
def draw_axes_from_ax3(ax3, viewer, scale=500, name=""):
    origin = ax3.Location()
    x_dir = ax3.XDirection()
    y_dir = ax3.YDirection()
    z_dir = ax3.Direction()

    draw_axis_arrow(viewer, origin, x_dir, scale=scale, name=name + "_X")
    draw_axis_arrow(viewer, origin, y_dir, scale=scale, name=name + "_Y")
    draw_axis_arrow(viewer, origin, z_dir, scale=scale, name=name + "_Z")

In [18]:
if __name__ == "__main__":

    step_path = "../data/0444-1 ANGLED.step"
    # step_path = "../data/ncTest.step"
    # step_path = "../data/TestEA.step" 
    # step_path = "../data/TestEAMirror.step"
    # step_path = "../data/TestUEA.step"
    # step_path = "../data/TestUEAMirror.step"
    # step_path = "../data/TestPFC.step"
    
    # Load shape and compute OBB 
    shape_orig = load_step(step_path)
    # obb_orig, obb_geom_orig = compute_obb(shape_orig)

    # #Display origin axis
    show_origin()

    # Asume worst case and no geometry aligns, force alignment
    # obb_aligned, obb_geom_aligned = compute_obb(aligned_shape)
    aligned_shape, trsf, cs = robust_align_solid_from_geometry(shape_orig)
    obb_ax3, obb_geom = compute_axis_aligned_bounding_box(aligned_shape)
    
    visualize(aligned_shape, viewer, color="PSS_LIGHT_BLUE")
    draw_obb_box(obb_geom, viewer, color="BLACK")
    # draw_axes_from_ax3(cs, viewer, name="GLOBAL")
    # draw_axes_from_ax3(obb_ax3, viewer, name="OBB")
    
    # # Create transform on the obb_aligned
    # orig_transform = obb_transform(obb_geom_aligned)    

    # # Apply translation to align OBB CS with Global CS
    # transformer = BRepBuilderAPI_Transform(aligned_shape, orig_transform, True, True)
    # shape_trans = transformer.Shape()
    # obb_trans, obb_geom_trans = compute_obb(shape_trans)
    
    # # Original shape
    # visualize(shape_orig, viewer, "RED")
    # draw_obb_box(obb_geom_orig, viewer, color="ORANGE")
    # draw_obb_axes(obb_orig, viewer, scale=500)
    
    # # Translated shape
    # visualize(shape_trans, viewer, color="BLUE")
    # draw_obb_box(obb_geom_trans, viewer, color="YELLOW")
    # draw_obb_axes(obb_trans, viewer, scale=500)

# debug ------------------------------------
    
    # print("Is aligned shape same as original:", aligned_shape.IsEqual(shape_orig))
    # # Manually apply transform for debug
    # test_shape = BRepBuilderAPI_Transform(shape_orig, trsf, True).Shape()
    # visualize(test_shape, viewer, "CYAN")

    # draw_axes_from_ax3(cs, viewer, 500, name="TO")  # Global
    # # draw_axes_from_ax3(from_cs, viewer, 1000,name="FROM")  # Original solid CS

    # _, new_obb_geom = compute_axis_aligned_bounding_box(aligned_shape)
    # draw_obb_box(new_obb_geom, viewer, color="YELLOW")
    
# debug ------------------------------------

    
    # fingerprint shape for classification
    width, height, area, face, centroid = fingerprint_shape(aligned_shape, obb_geom)
    print(f"Width: {width:.3f}, Height: {height:.3f}, CSA: {area:.3f}")

    # classify against JSON library
    # build classifiction input
    classifier_input_data = build_classification_input(obb_geom, width, height, area)
    result = classify_profile(classifier_input_data, json_path, tol_dim=1.0, tol_area=5)

    print(result)
    
    if result['Swapped_dimensions']:
    ## Rotating solid 

        print("Rotating Shape")
        width, height = height, width
        result["Measured_height"] == height,
        result["Measured_width"] == width,
        
        # print(f"\nRevised Height : {height}, Revised Width : {width}")
        # # Rotate shape about z axis
        # shape_rotated = rotate_z90(shape_trans)
        # obb_rotated, obb_geom_rotated = compute_obb(shape_rotated)

        # # Show Rotated Shape
        # visualize(shape_rotated, viewer, color="GREEN")
        # draw_obb_box(obb_geom_rotated, viewer, color="CYAN")
        # draw_obb_axes(obb_rotated, viewer, scale=500)

    print_result_table(result)

Many colors for color name BLUE, using first.
Width: 203.600, Height: 203.200, CSA: 5873.149
{'mass': 46.1, 'height': 203.2, 'width': 203.6, 'csa': 5870.0, 'web_thickness': 7.2, 'flange_thickness': 11.0, 'root_radius': 10.2, 'toe_radius': 0.0, 'code_profile': 'I', 'Designation': '203x203x46', 'Category': 'UC', 'Measured_height': 203.20000020000043, 'Measured_width': 203.6000002000003, 'Measured_area': 5873.148700320526, 'Measured_length': 2280.000000200001, 'Match_score': 0.05364095060595109, 'Profile_type': 'I', 'Swapped_dimensions': False}


,Value
mass,46.1
height,203.2
width,203.6
csa,5870.0
web_thickness,7.2
flange_thickness,11.0
root_radius,10.2
toe_radius,0.0
code_profile,I
Designation,203x203x46


In [19]:
# analysis and positioning of nc start point per face

DSTV_FACE_MAP = {'I':['O','U','V'], 
                 'U':['H','U','O'], 
                 'L':['H','U']}

Below is the work on dstv origin point definition for the three different shapes.

In [20]:
# Determine channel and angle orientation based on centroid comparison
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop_SurfaceProperties

def shape_direction(face, shape_centroid):
    props = GProp_GProps()
    brepgprop.SurfaceProperties(face, props)
    face_centroid = props.CentreOfMass()

    flange_vec = gp_Vec(shape_centroid, face_centroid)
    # Determine main axis of deviation
    components = [abs(flange_vec.X()), abs(flange_vec.Y()), abs(flange_vec.Z())]
    dominant_axis = components.index(max(components))
    direction = "X" if dominant_axis == 0 else "Y" if dominant_axis == 1 else "Z"
    sign = 1 if flange_vec.Coord()[dominant_axis] > 0 else -1

    return direction, sign
    
if result['Profile_type'] != 'I':
    direction, sign = shape_direction(face, centroid)
    print(f"Profile type {result['Profile_type']} opens along {direction}, direction: {'+ve' if sign > 0 else '-ve'}")
    

In [28]:
# Channel DSTV origin based on OBB limits and sign direction of flange

def pick_channel_corner_from_obb(obb, sign):
    center = obb['center']
    half_x = obb['half_x']
    half_y = obb['half_y']
    half_z = obb['half_z']

    cx, cy, cz = center.X(), center.Y(), center.Z()

    if sign < 0:
        # Flange opens -Y → corner at min X, max Y
        x = cx - half_x
        y = cy + half_y
    else:
        # Flange opens +Y → corner at max X, min Y
        x = cx + half_x
        y = cy - half_y

    # You can choose to place this at start or end of beam length
    z = cz - half_z  # for example: start of beam

    return gp_Pnt(x, y, z)

if result['Profile_type'] == 'U':
    # construct point
    dstv_origin_channel = pick_channel_corner_from_obb(obb_geom, sign)
    
    # Visualize
    sphere = BRepPrimAPI_MakeSphere(dstv_origin_channel, 5).Shape()
    visualize(sphere, viewer, "RED")

In [29]:
# EA / UEA DSTV origin based on OBB limits and sign direction of flange

# check if we are working with EA or UEA
def is_equal_angle(width, height, tolerance_ratio=0.01):
    """
    Returns True if the profile is an equal angle based on width-height comparison.
    """
    return abs(width - height) < tolerance_ratio * max(width, height)

equal_angle = is_equal_angle(width, height) #true or false

# get OBB size for leg length
def get_obb_leg_lengths(obb):
    x_leg = 2 * obb['half_x']
    y_leg = 2 * obb['half_y']
    return x_leg, y_leg

# determine if longer leg is x or y
def classify_angle_from_obb(obb, tolerance_ratio=0.01):
    x_leg, y_leg = get_obb_leg_lengths(obb)
    is_equal = abs(x_leg - y_leg) < tolerance_ratio * max(x_leg, y_leg)
    longer_leg = "X" if x_leg > y_leg else "Y"
    return is_equal, longer_leg

# determine z point depending on longer leg position, longer leg should be horizontal, pointing left
def compute_z_origin_for_angle(obb, is_equal_angle, longer_leg):
    cz = obb['center'].Z()
    hz = obb['half_z']

    if is_equal_angle or longer_leg == "Y":
        return cz - hz  # Start of beam
    else:
        return cz + hz  # End of beam (UEA with horizontal long leg)


def compute_angle_corner_point(obb, direction, sign, is_equal_angle, longer_leg):
    cx, cy, cz = obb.Center().X(), obb.Center().Y(), obb.Center().Z()
    hx, hy = obb['half_x'], obb['half_y']

    if direction == "Y":
        x = cx - hx if sign > 0 else cx + hx
        y = cy + hy if sign > 0 else cy - hy
    elif direction == "X":
        x = cx + hx if sign > 0 else cx - hx
        y = cy - hy if sign > 0 else cy + hy
    else:
        raise NotImplementedError("Z-directed L profiles not yet supported")

    z = compute_z_origin_for_angle(obb, is_equal_angle, longer_leg)

    return gp_Pnt(x, y, z)

if result['Profile_type'] == 'L':
    
    is_equal_angle, longer_leg = classify_angle_from_obb(obb_geom)
    dstv_origin_angle = compute_angle_corner_point(obb_geom, direction, sign, is_equal_angle, longer_leg)
    
    sphere = BRepPrimAPI_MakeSphere(dstv_origin_angle, 5).Shape()
    visualize(sphere, viewer, "RED")

In [31]:
# Beam DSTV origin based on OBB limits and sign direction of flange

def compute_i_beam_corner_origin_and_cs(obb):
    # Extract OBB center and half-sizes
    cx, cy, cz = obb['center'].X(), obb['center'].Y(), obb['center'].Z()
    hx, hy, hz = obb['half_x'], obb['half_y'], obb['half_z']

    # LEFT = min X, BOTTOM = min Y, END = max Z
    x = cx - hx
    y = cy - hy
    z = cz + hz  # end of the beam

    origin = gp_Pnt(x, y, z)

    # # Axes from OBB
    # x_dir = gp_Dir(obb.XDirection())  # across flange (left to right)
    # y_dir = gp_Dir(obb.YDirection())  # up web (bottom to top)
    # z_dir = gp_Dir(obb.ZDirection())  # beam length

    # # Local CS: origin at bottom-left corner of bottom flange, facing along beam
    # cs = gp_Ax3(origin, z_dir, x_dir)

    # return origin, cs
    return origin

if result['Profile_type'] == 'I':
    # origin, cs = compute_i_beam_corner_origin_and_cs(obb)
    dstv_origin_beam = compute_i_beam_corner_origin_and_cs(obb_geom)
    
    sphere = BRepPrimAPI_MakeSphere(dstv_origin_beam, 5).Shape()
    visualize(sphere, viewer, "RED")
    